In [ ]:
import os
from PIL import Image
from instructg2i import image_grid, InstructG2IMultiGuidePipeline, get_neighbor_transforms


### Mix Style (CFG based)

In [2]:
device = 'cuda:6'
model_dir = "PeterJinGo/VirtualArtist"

In [ ]:
resolution=256
neighbor_num = 5
pipeline = InstructG2IMultiGuidePipeline.from_pretrained(model_dir, neighbor_num, device=device)

In [4]:
path1 = "/shared/data/bowenj4/opensource/InstructG2I/examples/monet_pictures"
path2 = "/shared/data/bowenj4/opensource/InstructG2I/examples/children_pictures"

In [ ]:
# multi image generation
text_prompt = 'a house in the snow'  # a man playing soccer, a man playing piano
scale_as = [0, 3, 10]
scale_bs = [0, 5, 15]

# graph2image pipeline
neighbor_transforms = get_neighbor_transforms(resolution)
neighbor_images = [[neighbor_transforms(Image.open(os.path.join(path1, n_file)).convert("RGB")) for n_file in os.listdir(path1)],
                   [neighbor_transforms(Image.open(os.path.join(path2, n_file)).convert("RGB")) for n_file in os.listdir(path2)]]
neighbor_masks = [[1,1,1,1,1],
                   [1,1,1,1,1]]

image_gens = []
for scale_a in scale_as:
    for scale_b in scale_bs:
        graph_guidance_scales = [scale_a, scale_b]

        image_gen = pipeline(prompt=text_prompt, 
                                        neighbor_images=neighbor_images, 
                                        neighbor_masks=neighbor_masks, 
                                        graph_guidance_scales=graph_guidance_scales,
                                        num_inference_steps=100).images[0]
        image_gens.append(image_gen)

print(f'Generated Image Monet, {scale_as} & {scale_bs}, {text_prompt}')
res_grid = image_grid(image_gens, len(scale_as), len(scale_bs))
res_grid.show()